# XML einlesen und zu einfachem JSON konvertieren

In [360]:
import json
import unicodedata
import xml.etree.ElementTree as ET

In [361]:
file_name = '209'

In [362]:
tree = ET.parse(f'res/{file_name}.xml')
root = tree.getroot()

In [363]:
sitzungsverlauf = root.find('sitzungsverlauf')

sitzungsbeginn = sitzungsverlauf.find('sitzungsbeginn')
tagesordnungspunkte = sitzungsverlauf.findall('tagesordnungspunkt')

In [364]:
all_speaks = []

Den Einleitenden Präsidenten und seinen Text extrahieren

In [365]:
main_president = sitzungsbeginn.find('name').text[:-1]

paragraphs =  [p.text for p in sitzungsbeginn.findall('p')]
text = ' '.join(p for p in paragraphs).strip()

all_speaks.append({
    'speaker': main_president,
    'text': text
})

Alle aus Tagesordnungspunkte und innerhalb von Reden dieser extrahieren und zeitlich geordnen bezüglich Redner speichern

In [366]:
def extract_rede(rede, main_president, data, start_with_president=False):
    president_turn = start_with_president
    speaker_turn = False

    speaker = ''
    fraktion = ''
    president = main_president

    texts = []

    for elem in rede.findall('./'):

        if elem.tag == 'kommentar':
            continue
            
        if elem.tag == 'rede':
            if len(texts) > 0:
                person = speaker if speaker_turn else president
                data.append({'speaker': person, 'party': fraktion, 'text': ' '.join(texts).strip()})
            texts = []

            data = extract_rede(elem, main_president, data)

        elif elem.tag == 'name':
            if len(texts) > 0:
                if speaker_turn:
                    data.append({'speaker': speaker, 'party': fraktion, 'text': ' '.join(texts).strip()})
                else:
                    data.append({'speaker': president, 'text': ' '.join(texts).strip()})
                texts = []
            
            president_turn = True
            speaker_turn = False
            president = elem.text[:-1]

        elif elem.tag == 'p' and elem.attrib['klasse'] == 'redner':
            if len(texts) > 0:
                if president_turn:
                    data.append({
                        'speaker': president,
                        'text': ' '.join(texts).strip(),
                    })
                else:
                    data.append({
                        'speaker': speaker,
                        'party': fraktion,
                        'text': ' '.join(texts).strip(),
                    })
                texts = []
            
            speaker_turn = True
            president_turn = False
            
            name = elem.find('redner').find('name')
            first_name = name.find('vorname').text
            last_name = name.find('nachname').text
            fraktion = name.find('fraktion').text if name.find('fraktion') is not None else ''
            speaker = f"{first_name} {last_name}"

        else:
            texts.append(elem.text)
    
    if len(texts) > 0:
        
        if president_turn:
            data.append({
                'speaker': president,
                'text': ' '.join(texts).strip(),
            })
        elif speaker_turn:
            
            data.append({
                'speaker': speaker,
                'party': fraktion,
                'text': ' '.join(texts).strip(),
            })

    return data

Iteriere über alle Tagesordnungspunkte und speichere die Redeteile in einer Variable

In [367]:
for tagesordnungspunkt in tagesordnungspunkte:
    all_speaks = extract_rede(tagesordnungspunkt, main_president, all_speaks, start_with_president=True)

Speichere das Ergebnis in einer JSON-Datei

In [368]:
with open(f'output/speakers_{file_name}.json', 'w', encoding='utf-8') as f_out:
    json.dump(all_speaks, f_out, ensure_ascii=False)